This example builds hdf files to store network information.  For each stream unit a list of parent segment ids are stored, along with local and network areas.  These variables can then be easily accessed to calculate upstream summaries without rebuilding the stream network for each calculation.  This example works with the NHDPlusV2.1 dataset but has been tested on the NHDPlusHR as well.

In [1]:
import pandas as pd

In [2]:
stream_df = pd.read_pickle('data/flow_table_sub.pkl')

In [3]:
#Use To From Nodes to build table of all seg_ids and upstream seg ids.  Note many seg_ids will have multiple records, one for each upstream seg_id
upid_temp = stream_df[['seg_id','ToNode']]
upstream_seg_list = pd.merge(stream_df, upid_temp, left_on='FromNode', right_on='ToNode', how='left')

#Rename fields (note if not using NHDPlusV2.1 files make sure you name files accordingly)
upstream_seg_list.rename(columns={'seg_id_x':'seg_id','seg_id_y':'upstream_seg_id','LENGTHKM':'len_km','AreaSqKM':'area_sqkm', 'TotDASqKM':'tot_area_sqkm', 'RPUID': 'rpuid', 'VPUID':'vpuid'}, inplace=True)

#Drop unneeded fields
upstream_seg_list.drop(columns=['FromNode','ToNode_x','ToNode_y'], inplace=True)

#Print shape of dataframe to get an idea of if we have the right output or not
#upstream_seg_list.shape

In [4]:
#Fill null upstream segment ids with zeros
isnull_df = (upstream_seg_list[upstream_seg_list['upstream_seg_id'].isnull()])
isnull_df.fillna(0, inplace=True)

isnotnull_df = (upstream_seg_list[upstream_seg_list['upstream_seg_id'].notna()])
upstream_seg_list = isnotnull_df.append(isnull_df)

#upstream_seg_list.shape

C:\Users\dwieferich\AppData\Local\Continuum\Anaconda3\lib\site-packages\pandas\core\frame.py:4034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [5]:
#Delete unneeded dataframes and data
del isnull_df
del isnotnull_df
del upid_temp
del stream_df

In [6]:
#There is an issue with a to/from node in NHDPlusV2, this ensures the bad (duplicated) node doesn;t result in false routing
upstream_seg_list.shape
del_rows = upstream_seg_list[((upstream_seg_list['seg_id']==24561351.0) & (upstream_seg_list['upstream_seg_id']==22227020.0)) | ((upstream_seg_list['seg_id']==24561351.0) & (upstream_seg_list['upstream_seg_id']==23949489.0))].index
upstream_seg_list.drop(del_rows, inplace=True)

In [7]:
upstream_seg_list.columns

Index(['seg_id', 'COMID', 'FTYPE', 'area_sqkm', 'StreamOrde', 'tot_area_sqkm',
       'REACHCODE', 'len_km', 'StreamOrde', 'StartFlag', 'FromMeas', 'ToMeas',
       'watershed_id', 'proc_unit', 'upstream_seg_id'],
      dtype='object')

In [8]:
#Run Build Network by region, uses floats for ids
import time
import xstrm.build_network_rpu as build

start_time = time.clock()

watersheds = ['01','02','03','04','09','11','12','13','15','16','17','18']

for watershed in watersheds:
    tic = time.clock()
    #subset dataframe by drainage
    df_infile = upstream_seg_list.loc[upstream_seg_list['watershed_id']==watershed]
    #print (df_infile.shape)

    traverse_queue = build.upstream_setup(df_infile)
    build.upstream_build_network(traverse_queue)
    toc = time.clock()
    print ('region' +str(watershed) + ':' + str(toc-tic))

end_time = time.clock()
print ('total seconds: ' + (str(end_time-start_time)))

region01:289.4623286222535
region02:545.8974282580187
region03:1453.377068238939
region04:429.15438620546774
region09:120.49025267414163
region11:6974.4573935120425
region12:284.37484410259276
region13:262.3691435452274
region15:881.161270853785
region16:401.33782257088205
region17:1107.0590225761316
region18:600.8853309838705
total seconds: 13350.032464222


In [ ]:
upstream_seg_list.columns

#Seconds to process each region when processing entire region.  Although this is faster to build hdf5 files upstream agg calculations are slower due to larger hdf5 file size (entire watershed vs. regional processing unit)

region11:3031.6833874993254
region14:313.4585673921697
region1:70.33057106223168
region2:143.75602766841666
region3:399.6618969961214
region4:106.68949977171542
region9:41.31301464378066
region12:73.93708053568935
region13:81.08523266821976
region16:95.8744506211824
region17:297.7542863581921
region18:148.6522641773645

In [ ]:

#Shows number of records per watershed 
l = upstream_seg_list['proc_unit'].tolist()
import collections
counter=collections.Counter(l)
print(counter)

testing code when create hdf5 by regional processing unit (in this test the file opens and closes files every record).  it creates many hdf5 files and takes longer to build, but upstream calc performance is improved greatly

region01:289.4623286222535
region02:545.8974282580187
region03:1453.377068238939
region04:429.15438620546774
region09:120.49025267414163
region11:6974.4573935120425
region12:284.37484410259276
region13:262.3691435452274
region15:881.161270853785
region16:401.33782257088205
region17:1107.0590225761316
region18:600.8853309838705
total seconds: 13350.032464222